# Analysis of Reversi Data

In [1]:
import struct
import glob
from typing import List, Tuple, Dict

In [2]:
NUM_OF_READ = 20
CUT_THRESHOLD = 50

In [3]:
def load_wtb_file(name: str) -> List[Tuple[int, List[Tuple[int]]]]:
    with open(name, "rb") as file:
        headers = file.read(16)
        (year1, year2, month, day, record_num, _, year, size, record_type, depth, _) = struct.unpack("BBBBIHHBBBB", headers)

        record = []
        for _ in range(record_num):
            (contest_id, black_id, white_id, black_num, black_best) = struct.unpack("HHHBB", file.read(8))
            data = []
            for _ in range(60):
                data.append(struct.unpack("B", file.read(1))[0])
            record.append((black_best, data))
        return record

In [4]:
data = []
for file in glob.glob("./data/*.wtb"):
    data += load_wtb_file(file)

In [5]:
def concat_moves(moves: List[int]) -> str:
    return ",".join(map(str, moves))

def split_moves(moves: str) -> List[int]:
    return list(map(int, moves.split(",")))

In [6]:
assert split_moves(concat_moves([1, 2, 3])) == [1, 2, 3]

In [7]:
Data = Dict[Tuple[int, str], Dict[int, Tuple[int, float]]]

In [8]:
move_lists: Data = { }
for i in range(1, NUM_OF_READ):
    for (best, move) in data:
        key = i, concat_moves(move[0:i])
        if key not in move_lists:
            move_lists[key] = { }
        if move[i] in move_lists[key]:
            (count, score) = move_lists[key][move[i]]
            move_lists[key][move[i]] = (count + 1, score + best)
        else:
            move_lists[key][move[i]] = (1, best)

In [9]:
move_lists

{(1, '56'): {66: (5936, 192682), 64: (11259, 364343), 46: (325, 13896)},
 (2, '56,66'): {65: (5761, 188773),
  76: (33, 817),
  34: (64, 1432),
  43: (78, 1660)},
 (2, '56,64'): {53: (3044, 97310),
  33: (6221, 205724),
  43: (1760, 54733),
  63: (192, 5610),
  73: (42, 966)},
 (2, '56,46'): {35: (259, 11645),
  37: (20, 726),
  34: (12, 480),
  33: (20, 639),
  36: (14, 406)},
 (3, '56,66,65'): {46: (5761, 188773)},
 (3, '56,64,53'): {46: (2918, 91444),
  66: (95, 4641),
  62: (14, 540),
  42: (17, 685)},
 (3, '56,64,33'): {34: (5681, 185648),
  46: (347, 12457),
  36: (79, 3553),
  57: (114, 4066)},
 (3, '56,64,43'): {34: (1363, 42039),
  46: (148, 5569),
  32: (19, 574),
  57: (204, 5445),
  36: (26, 1106)},
 (3, '56,46,35'): {26: (35, 1698),
  66: (109, 4886),
  64: (87, 3771),
  24: (28, 1290)},
 (3, '56,64,63'): {46: (159, 4416), 66: (23, 810), 62: (10, 384)},
 (3, '56,64,73'): {74: (11, 349), 46: (22, 437), 36: (6, 94), 57: (3, 86)},
 (3, '56,46,37'): {65: (4, 115),
  67: (1, 40

In [10]:
for num, move in move_lists.keys():
    for m in move_lists[num, move].keys():
        (count, score) = move_lists[num, move][m]
        move_lists[num, move][m] = (count, score / count)

In [11]:
move_lists

{(1, '56'): {66: (5936, 32.45990566037736),
  64: (11259, 32.360156319388935),
  46: (325, 42.75692307692308)},
 (2, '56,66'): {65: (5761, 32.76740149279639),
  76: (33, 24.757575757575758),
  34: (64, 22.375),
  43: (78, 21.28205128205128)},
 (2, '56,64'): {53: (3044, 31.967805519053876),
  33: (6221, 33.06928146600225),
  43: (1760, 31.098295454545454),
  63: (192, 29.21875),
  73: (42, 23.0)},
 (2, '56,46'): {35: (259, 44.96138996138996),
  37: (20, 36.3),
  34: (12, 40.0),
  33: (20, 31.95),
  36: (14, 29.0)},
 (3, '56,66,65'): {46: (5761, 32.76740149279639)},
 (3, '56,64,53'): {46: (2918, 31.33790267306374),
  66: (95, 48.85263157894737),
  62: (14, 38.57142857142857),
  42: (17, 40.294117647058826)},
 (3, '56,64,33'): {34: (5681, 32.67875374053864),
  46: (347, 35.89913544668588),
  36: (79, 44.9746835443038),
  57: (114, 35.666666666666664)},
 (3, '56,64,43'): {34: (1363, 30.842993396918562),
  46: (148, 37.62837837837838),
  32: (19, 30.210526315789473),
  57: (204, 26.69117647

In [12]:
move_table = {}
for num, move in move_lists.keys():
    is_black = num % 2 == 0
    best_move = 0
    best_score = -1000 if is_black else 1000
    appeard = 0

    for m in move_lists[num, move].keys():
        (count, score) = move_lists[num, move][m]
        appeard += count
        if is_black:
            if score  > best_score:
                best_score = score
                best_move = m
        else:
            if score < best_score:
                best_score = score
                best_move = m

    if appeard >= CUT_THRESHOLD:
        move_table[num, move] = best_move

In [13]:
move_table

{(1, '56'): 64,
 (2, '56,66'): 65,
 (2, '56,64'): 33,
 (2, '56,46'): 35,
 (3, '56,66,65'): 46,
 (3, '56,64,53'): 46,
 (3, '56,64,33'): 34,
 (3, '56,64,43'): 57,
 (3, '56,46,35'): 64,
 (3, '56,64,63'): 46,
 (3, '56,66,34'): 57,
 (3, '56,66,43'): 57,
 (4, '56,66,65,46'): 77,
 (4, '56,64,53,46'): 35,
 (4, '56,64,33,34'): 43,
 (4, '56,64,33,46'): 53,
 (4, '56,64,33,36'): 34,
 (4, '56,64,53,66'): 65,
 (4, '56,64,43,34'): 65,
 (4, '56,64,63,46'): 65,
 (4, '56,64,43,46'): 53,
 (4, '56,46,35,66'): 34,
 (4, '56,64,33,57'): 67,
 (4, '56,46,35,64'): 36,
 (4, '56,64,43,57'): 63,
 (5, '56,66,65,46,57'): 75,
 (5, '56,64,53,46,35'): 67,
 (5, '56,64,33,34,43'): 32,
 (5, '56,66,65,46,67'): 74,
 (5, '56,66,65,46,35'): 74,
 (5, '56,64,53,66,65'): 62,
 (5, '56,64,43,34,53'): 42,
 (5, '56,64,53,46,74'): 57,
 (5, '56,64,43,34,33'): 46,
 (5, '56,66,65,46,34'): 63,
 (5, '56,64,63,46,65'): 57,
 (5, '56,64,43,46,66'): 57,
 (5, '56,64,43,34,65'): 46,
 (5, '56,66,65,46,33'): 74,
 (5, '56,46,35,66,34'): 25,
 (5, '

In the original data, the first move is fixed to `56`. We are going to flip boards to make it easy to search a best move.

In [14]:
move_table_complete = { }
for num, move in move_table.keys():
    moves = split_moves(move)
    best_move = move_table[(num, move)]

    move_table_complete[(num, move)] = best_move

    def flip1(x: int) -> int:
        return (x % 10) * 10 + (x // 10)
    def flip2(x: int) -> int:
        return (9 - x // 10) * 10 + (9 - x % 10)
    def flip3(x: int) -> int:
        return flip2(flip1(x))

    move_table_complete[(num, concat_moves(list(map(flip1, moves))))] = flip1(best_move)
    move_table_complete[(num, concat_moves(list(map(flip2, moves))))] = flip2(best_move)
    move_table_complete[(num, concat_moves(list(map(flip3, moves))))] = flip3(best_move)

In [15]:
move_table_complete

{(1, '56'): 64,
 (1, '65'): 46,
 (1, '43'): 35,
 (1, '34'): 53,
 (2, '56,66'): 65,
 (2, '65,66'): 56,
 (2, '43,33'): 34,
 (2, '34,33'): 43,
 (2, '56,64'): 33,
 (2, '65,46'): 33,
 (2, '43,35'): 66,
 (2, '34,53'): 66,
 (2, '56,46'): 35,
 (2, '65,64'): 53,
 (2, '43,53'): 64,
 (2, '34,35'): 46,
 (3, '56,66,65'): 46,
 (3, '65,66,56'): 64,
 (3, '43,33,34'): 53,
 (3, '34,33,43'): 35,
 (3, '56,64,53'): 46,
 (3, '65,46,35'): 64,
 (3, '43,35,46'): 53,
 (3, '34,53,64'): 35,
 (3, '56,64,33'): 34,
 (3, '65,46,33'): 43,
 (3, '43,35,66'): 65,
 (3, '34,53,66'): 56,
 (3, '56,64,43'): 57,
 (3, '65,46,34'): 75,
 (3, '43,35,56'): 42,
 (3, '34,53,65'): 24,
 (3, '56,46,35'): 64,
 (3, '65,64,53'): 46,
 (3, '43,53,64'): 35,
 (3, '34,35,46'): 53,
 (3, '56,64,63'): 46,
 (3, '65,46,36'): 64,
 (3, '43,35,36'): 53,
 (3, '34,53,63'): 35,
 (3, '56,66,34'): 57,
 (3, '65,66,43'): 75,
 (3, '43,33,65'): 42,
 (3, '34,33,56'): 24,
 (3, '56,66,43'): 57,
 (3, '65,66,34'): 75,
 (3, '43,33,56'): 42,
 (3, '34,33,65'): 24,
 (4,

In [16]:
with open("preprocessed.txt", "w") as file:
    for key in move_table_complete.keys():
        for move in split_moves(key[1]):
            pos = chr(ord('A') + move // 10 - 1) + str(move % 10)
            file.write(f"{pos}")
        move = move_table_complete[key]
        pos = chr(ord('A') + move // 10 - 1) + str(move % 10)
        file.write(f" {pos}\n")